**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

[11:50:51] Initializing Normalizer


**Set up**
- **software**: The path to the software folder.
- **proteinfile**: The path to the protein file (.pdb).
- **pocket**: The method to use for pocket determination. Must be one of 'reference' or 'dogsitescorer'.
- **dockinglibrary: The path to the docking library file (.sdf).
- **idcolumn**: The unique identifier column used in the docking library.
- **protonation**: The method to use for compound protonation. Must be one of 'pkasolver', 'GypsumDL', or 'None'.
- **docking**: The method(s) to use for docking. Must be one or more of 'GNINA', 'SMINA', or 'PLANTS'.
- **metric**: The method(s) to use for pose clustering. Must be one or more of 'RMSD', 'spyRMSD', 'espsim', 'USRCAT', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', or 'bestpose_PLANTS'.
- **nposes**: The number of poses to generate for each docking software. Default=10
- **exhaustiveness**: The precision used if docking with SMINA/GNINA. Default=8
- **parallel**: Whether or not to run the workflow in parallel. Default=1 (on). Can be set to 1 (on) or 0 (off).
- **ncpus**: The number of cpus to use for the workflow. Default behavior is to use half of the available cpus.
- **clustering**: Which algorithm to use for clustering. Must be one of 'KMedoids', 'Aff_prop'.
- **rescoring**: Which scoring functions to use for rescoring. Must be one or more of 'gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'vinardo', 'plp', 'AAScore'.

The software will then create a temporary directory to store the output of the various functions.

In [2]:
software = Path('/home/mario/DockM8/software')
protein_file = Path('/home/mario/LitPCBA/ESR1_ago/1l2i.pdb')
ref_file = Path('/home/mario/LitPCBA/ESR1_ago/1l2i_ligand.sdf')
pocket = 'reference'
protonation = 'pkasolver'
docking_library = Path('/home/mario/LitPCBA/ESR1_ago/merged.sdf')
docking_programs = ['GNINA', 'SMINA', 'PLANTS', 'QVINAW', 'QVINA2']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', 'bestpose', 'bestpose_PLANTS', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_QVINA2', 'bestpose_QVINAW']
clustering_method = 'KMedoids'
rescoring= ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'RTMScore', 'vinardo', 'SCORCH', 'plecscore', 'nnscore']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
ncpus = 22
#Create a temporary folder for all further calculations
w_dir = protein_file.parent
print('The working directory has been set to:', w_dir)
(w_dir / 'temp').mkdir(parents=True, exist_ok=True)
if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, protein_file, 8)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')


The working directory has been set to: /home/mario/LitPCBA/ESR1_ago

[2023-Jun-28 11:50:54]: Extracting pocket from /home/mario/LitPCBA/ESR1_ago/1l2i.pdb using /home/mario/LitPCBA/ESR1_ago/1l2i_ligand.sdf as reference ligand

[2023-Jun-28 11:50:56]: Finished extracting pocket from /home/mario/LitPCBA/ESR1_ago/1l2i.pdb using /home/mario/LitPCBA/ESR1_ago/1l2i_ligand.sdf as reference ligand
{'center': [5.48, -0.05, -5.55], 'size': [16.0, 16.0, 16.0]}


**Pocket Extraction**  

This cell will extract the pocket based on the method specified in the 'pocket' variable. Using 'reference' will use the reference ligand to define the pocket. Using 'dogsitescore' will query the dogsitescorer server and use the pocket with the largest volume.

In [3]:
if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, protein_file, 10)
    print(pocket_definition)
if pocket == 'RoG':
    pocket_definition = get_pocket_RoG(ref_file, protein_file)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')


[2023-Jun-28 11:50:57]: Extracting pocket from /home/mario/LitPCBA/ESR1_ago/1l2i.pdb using /home/mario/LitPCBA/ESR1_ago/1l2i_ligand.sdf as reference ligand

[2023-Jun-28 11:50:59]: Finished extracting pocket from /home/mario/LitPCBA/ESR1_ago/1l2i.pdb using /home/mario/LitPCBA/ESR1_ago/1l2i_ligand.sdf as reference ligand
{'center': [5.48, -0.05, -5.55], 'size': [20.0, 20.0, 20.0]}


**Library preparation**  

In [4]:
if (w_dir / 'temp' / 'final_library.sdf').is_file() == False:
    prepare_library(docking_library, id_column, software, protonation, ncpus)


[2023-Jun-28 11:51:01]: Calculating protonation states using pkaSolver...


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


COC=C(C(=O)OC)c1ccccc1Oc1cc(Oc2ccccc2C#N)ncn1

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

COC=C(C(=O)OC)c1ccccc1Oc1cc(Oc2ccccc2C#N)ncn1

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Proposed mol at pH 7.4: COC=C(C(=O)OC)c1ccccc1Oc1cc(Oc2ccccc2C#N)ncn1


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


CCCCOCCOCCOCc1cc2c(cc1CCC)OCO2

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

CCCCOCCOCCOCc1cc2c(cc1CCC)OCO2

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Proposed mol at pH 7.4: CCCCOCCOCCOCc1cc2c(cc1CCC)OCO2
#########################
Could not identify any ionizable group. Aborting.
#########################


...

CC(=Cc1ccccc1)[N+](=O)[O-]

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

CC(=Cc1ccccc1)[N+](=O)[O-]

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Proposed mol at pH 7.4: CC(=Cc1ccccc1)[N+](=O)[O-]
#########################
Could not identify any ionizable group. Aborting.
#########################


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


C=COCC1CCC(CO)CC1

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

C=COCC1CCC(CO)CC1

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Proposed mol at pH 7.4: C=COCC1CCC(CO)CC1


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


CCC(=CC(=NO)C(N)=O)[C@H](C)[N+](=O)[O-]

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

CCC(=CC(=NO)C(N)=O)[C@H](C)[N+](=O)[O-]

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Proposed mol at pH 7.4: CCC(=CC(=NO)C(N)=O)[C@H](C)[N+](=O)[O-]


**Docking**

This cell will dock all compounds in the receptor, using the reference ligand as a way to define the binding site. (Note: DogSiteScorer not yet implemented here).

The docking algorithms specified in the 'docking' variable will be used.

The docking will be done in on parallel CPU cores depending on the value or the 'parallel' variable.

The docking results are written to the temporary folder. 

In [ ]:
docking(w_dir, protein_file, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5)

All poses are then loaded into memory for clustering

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(str(w_dir / 'temp' / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

**Clustering**

This cell will perform the clustering according to the values of the 'clusering_metrics', 'clustering_method' and 'parallel' variables. If it detects that the clustering file for that metric has already been generated, it will skip it.

In [ ]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir/'temp'/'clustering/'f'{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

**Rescoring**

This cell will rescore all the clustered .sdf files according to the specified scoring functions and the value of the 'parallel' variable.

In [ ]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, pocket_definition, software, w_dir / 'temp' / 'clustering' / f'{metric}_clustered.sdf', rescoring_functions, ncpus)


**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
- *Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
- *Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
- *Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
- *Method 4* : Calculates the Rank by Rank consensus
- *Method 5* : Calculates the Rank by Vote consensus
- *Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
- *Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [ ]:
calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)
apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)